In [1]:
import tensorflow as tf
print(tf.__version__)

tf.enable_eager_execution()

1.14.0


In [2]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

In [4]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())
    
for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())

In [5]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [6]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [7]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

b this was soul provoking i am an iranian and living in <OOV> 21st century i didn't know that such big <OOV> have been living in such conditions at the time of my grandfather br br you see that today or even in <OOV> on one side of the world a lady or a baby could have everything served for him or her clean and on demand but here 80 years ago people <OOV> their life to go to somewhere with more grass it's really interesting that these <OOV> bear those difficulties to find <OOV> for their sheep but they lose many the sheep on their way br br i praise the americans who accompanied this tribe they were as
b"This was soul-provoking! I am an Iranian, and living in th 21st century, I didn't know that such big tribes have been living in such conditions at the time of my grandfather!<br /><br />You see that today, or even in 1925, on one side of the world a lady or a baby could have everything served for him or her clean and on-demand, but here 80 years ago, people ventured their life to go to

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') 
])

In [9]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - ETA: 6s - loss: 1.8403e-06 - acc: 1.000 - ETA: 4s - loss: 1.2739e-06 - acc: 1.000 - ETA: 4s - loss: 1.2469e-06 - acc: 1.000 - ETA: 3s - loss: 1.2761e-06 - acc: 1.000 - ETA: 3s - loss: 1.2231e-06 - acc: 1.000 - ETA: 3s - loss: 1.2297e-06 - acc: 1.000 - ETA: 3s - loss: 1.2149e-06 - acc: 1.000 - ETA: 3s - loss: 1.2156e-06 - acc: 1.000 - ETA: 3s - loss: 1.2162e-06 - acc: 1.000 - ETA: 3s - loss: 1.2275e-06 - acc: 1.000 - ETA: 3s - loss: 1.2184e-06 - acc: 1.000 - ETA: 3s - loss: 1.2159e-06 - acc: 1.000 - ETA: 3s - loss: 1.2007e-06 - acc: 1.000 - ETA: 3s - loss: 1.1891e-06 - acc: 1.000 - ETA: 3s - loss: 1.1907e-06 - acc: 1.000 - ETA: 3s - loss: 1.1947e-06 - acc: 1.000 - ETA: 3s - loss: 1.1988e-06 - acc: 1.000 - ETA: 3s - loss: 1.1935e-06 - acc: 1.000 - ETA: 3s - loss: 1.1924e-06 - acc: 1.000 - ETA: 3s - loss: 1.1906e-06 - acc: 1.000 - ETA: 3s - loss: 1.1846e-06 - acc: 1.000 - ETA: 3s - 

25000/25000 [==============================] - ETA: 4s - loss: 3.5949e-07 - acc: 1.000 - ETA: 4s - loss: 2.0337e-07 - acc: 1.000 - ETA: 4s - loss: 1.8058e-07 - acc: 1.000 - ETA: 4s - loss: 1.7434e-07 - acc: 1.000 - ETA: 3s - loss: 1.7751e-07 - acc: 1.000 - ETA: 3s - loss: 1.8068e-07 - acc: 1.000 - ETA: 3s - loss: 1.8844e-07 - acc: 1.000 - ETA: 3s - loss: 1.9201e-07 - acc: 1.000 - ETA: 3s - loss: 1.8972e-07 - acc: 1.000 - ETA: 3s - loss: 1.8460e-07 - acc: 1.000 - ETA: 3s - loss: 1.8719e-07 - acc: 1.000 - ETA: 3s - loss: 1.8408e-07 - acc: 1.000 - ETA: 3s - loss: 1.8215e-07 - acc: 1.000 - ETA: 3s - loss: 1.8355e-07 - acc: 1.000 - ETA: 3s - loss: 1.8184e-07 - acc: 1.000 - ETA: 3s - loss: 1.8160e-07 - acc: 1.000 - ETA: 3s - loss: 1.7884e-07 - acc: 1.000 - ETA: 3s - loss: 1.7793e-07 - acc: 1.000 - ETA: 3s - loss: 1.7718e-07 - acc: 1.000 - ETA: 2s - loss: 1.7703e-07 - acc: 1.000 - ETA: 2s - loss: 1.7905e-07 - acc: 1.000 - ETA: 2s - loss: 1.8022e-07 - acc: 1.000 - ETA: 2s - loss: 1.7845e-07 - 

In [11]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [12]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [13]:
try:
    from google.colab import files
except ImportError:
    pass
else:
    files.download('vecs.tsv')
    files.download('meta.tsv')

In [14]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1429], [968], [4], [1538], [1538], [4738], [], [790], [2013], [11], [2921], [2188], [], [790], [2013], [11], [579], [], [11], [579], [], [4], [1783], [4], [4508], [11], [2921], [1278], [], [], [2013], [1005], [2921], [968], [579], [790], []]


In [15]:
print("done with this project")

done with this project
